# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter, requests, math
from scipy.stats import percentileofscore as score

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data =requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'A(KANRGEEA) STSCSEOQ/ DL TBNLMGAL',
 'calculationPrice': 'tops',
 'open': None,
 'openTime': None,
 'openSource': 'lfcofiai',
 'close': None,
 'closeTime': None,
 'closeSource': 'ifafcilo',
 'high': None,
 'highTime': None,
 'highSource': None,
 'low': None,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 133.82,
 'latestSource': 'IEX real time price',
 'latestTime': '10:01:17 AM',
 'latestUpdate': 1643994594073,
 'latestVolume': None,
 'iexRealtimePrice': 132.185,
 'iexRealtimeSize': 102,
 'iexLastUpdated': 1648978130147,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 129.88,
 'previousVolume': 101822346,
 'change': 0.954,
 'changePercent': 0.00774,
 'volume': None,
 'iexMarketPercent': 0.0103570167048549,
 'iexVolume': 19

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
data['peRatio']

41.12

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_df = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data =requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_df = final_df.append(pd.Series([symbol,data[symbol]['quote']['latestPrice'],data[symbol]['quote']['peRatio'],'N/A'],index = my_columns),ignore_index=True)
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,119.400,54.14,N/A
1,AAL,16.838,-1.3,N/A
2,AAP,166.690,24.85,N/A
3,AAPL,131.500,40.43,N/A
4,ABBV,106.040,23.05,N/A
...,...,...,...,...
95,CINF,84.699,18,N/A
96,CL,87.371,28.71,N/A
97,CLX,211.600,23.22,N/A
98,CMA,55.040,15.06,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_df.sort_values('Price-to-Earnings Ratio',ascending = False, inplace =True)
final_df = final_df[:50]
final_df.reset_index(inplace = True, drop = True)
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,ADSK,302.640,160.17,N/A
1,AMD,100.250,133.03,N/A
2,CCI,161.664,113.33,N/A
3,ALK,52.480,109.55,N/A
4,AMZN,3265.900,97.21,N/A
5,BDX,256.044,92.34,N/A
6,AVGO,435.830,83.86,N/A
7,ANSS,355.720,79.51,N/A
8,ADBE,497.100,65.02,N/A
9,BKNG,2089.000,62.61,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = float(portfolio_size)/len(final_df)
for i in range(0,len(final_df)):
    final_df.loc[i,'Number of Shares to Buy'] = math.floor(position_size/final_df.loc[i, 'Price'])
    
final_df

C:\Users\q3604\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,ADSK,302.640,160.17,66
1,AMD,100.250,133.03,199
2,CCI,161.664,113.33,123
3,ALK,52.480,109.55,381
4,AMZN,3265.900,97.21,6
5,BDX,256.044,92.34,78
6,AVGO,435.830,83.86,45
7,ANSS,355.720,79.51,56
8,ADBE,497.100,65.02,40
9,BKNG,2089.000,62.61,9


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [11]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data =requests.get(batch_api_call_url).json()
data

#Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

#Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

#Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

#Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [12]:
rv_columns = ['Ticker',
              'Price',
              'Number of Shares to Buy',
              'Price-to-Earnings Ratio',
              'PE Percentile',
              'Price-to-Book Ratio',
              'PB Percentile',
              'Price-to-Sales Ratio',
              'PS Percentile',
              'EV/EBITDA',
              'EV/EBITDA Percentile',
              'EV/GP',
              'EV/GP Percentile',
              'RV Score']

rv_df = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data =requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
            
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        
        rv_df = rv_df.append(pd.Series([
            symbol,
            data[symbol]['quote']['latestPrice'],
            'N/A',
            data[symbol]['quote']['peRatio'],
            'N/A',
            data[symbol]['advanced-stats']['priceToBook'],
            'N/A',
            data[symbol]['advanced-stats']['priceToSales'],
            'N/A',
            ev_to_ebitda,
            'N/A',
            ev_to_gross_profit,
            'N/A',
            'N/A'
          ],index = rv_columns),ignore_index=True)
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,121.700,N/A,53.9,N/A,7.28,N/A,7.2,N/A,33.799598,N/A,13.074524,N/A,N/A
1,AAL,16.891,N/A,-1.3,N/A,-1.85,N/A,0.4177,N/A,-6.154534,N/A,1.357425,N/A,N/A
2,AAP,169.900,N/A,24.05,N/A,2.9,N/A,1.14,N/A,11.298304,N/A,2.450171,N/A,N/A
3,AAPL,135.090,N/A,41.07,N/A,33.92,N/A,8.06,N/A,28.156315,N/A,20.291113,N/A,N/A
4,ABBV,109.510,N/A,22.87,N/A,12.4,N/A,4.6,N/A,15.968320,N/A,9.395932,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,CINF,85.323,N/A,18,N/A,1.36,N/A,1.77,N/A,7.778563,N/A,1.684011,N/A,N/A
96,CL,90.004,N/A,28.24,N/A,114.24,N/A,4.59,N/A,19.301071,N/A,7.936676,N/A,N/A
97,CLX,208.840,N/A,22.82,N/A,23,N/A,3.7,N/A,18.161695,N/A,8.015534,N/A,N/A
98,CMA,53.700,N/A,14.46,N/A,1.0016,N/A,2.34,N/A,9.746571,N/A,2.713624,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [13]:
rv_df[rv_df.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,221.11,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,231.44,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [14]:
for column in ['Price-to-Earnings Ratio','Price-to-Book Ratio','Price-to-Sales Ratio','EV/EBITDA','EV/GP']:
    rv_df[column].fillna(rv_df[column].mean(), inplace = True)
    
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,121.700,N/A,53.90,N/A,7.2800,N/A,7.2000,N/A,33.799598,N/A,13.074524,N/A,N/A
1,AAL,16.891,N/A,-1.30,N/A,-1.8500,N/A,0.4177,N/A,-6.154534,N/A,1.357425,N/A,N/A
2,AAP,169.900,N/A,24.05,N/A,2.9000,N/A,1.1400,N/A,11.298304,N/A,2.450171,N/A,N/A
3,AAPL,135.090,N/A,41.07,N/A,33.9200,N/A,8.0600,N/A,28.156315,N/A,20.291113,N/A,N/A
4,ABBV,109.510,N/A,22.87,N/A,12.4000,N/A,4.6000,N/A,15.968320,N/A,9.395932,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,CINF,85.323,N/A,18.00,N/A,1.3600,N/A,1.7700,N/A,7.778563,N/A,1.684011,N/A,N/A
96,CL,90.004,N/A,28.24,N/A,114.2400,N/A,4.5900,N/A,19.301071,N/A,7.936676,N/A,N/A
97,CLX,208.840,N/A,22.82,N/A,23.0000,N/A,3.7000,N/A,18.161695,N/A,8.015534,N/A,N/A
98,CMA,53.700,N/A,14.46,N/A,1.0016,N/A,2.3400,N/A,9.746571,N/A,2.713624,N/A,N/A


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [15]:
rv_df[rv_df.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [16]:
metrics = {
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio': 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_df.index:
        rv_df.loc[row, metrics[metric]] = score(rv_df[metric], rv_df.loc[row, metric])/100

rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,121.700,N/A,53.90,0.87,7.2800,0.7,7.2000,0.8,33.799598,0.88,13.074524,0.78,N/A
1,AAL,16.891,N/A,-1.30,0.08,-1.8500,0.04,0.4177,0.03,-6.154534,0.03,1.357425,0.07,N/A
2,AAP,169.900,N/A,24.05,0.47,2.9000,0.34,1.1400,0.18,11.298304,0.29,2.450171,0.17,N/A
3,AAPL,135.090,N/A,41.07,0.77,33.9200,0.98,8.0600,0.84,28.156315,0.83,20.291113,0.92,N/A
4,ABBV,109.510,N/A,22.87,0.44,12.4000,0.82,4.6000,0.63,15.968320,0.49,9.395932,0.62,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,CINF,85.323,N/A,18.00,0.31,1.3600,0.18,1.7700,0.25,7.778563,0.14,1.684011,0.1,N/A
96,CL,90.004,N/A,28.24,0.54,114.2400,0.99,4.5900,0.62,19.301071,0.61,7.936676,0.49,N/A
97,CLX,208.840,N/A,22.82,0.43,23.0000,0.95,3.7000,0.53,18.161695,0.56,8.015534,0.5,N/A
98,CMA,53.700,N/A,14.46,0.21,1.0016,0.1,2.3400,0.385,9.746571,0.22,2.713624,0.18,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [17]:
from statistics import mean
for row in rv_df.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_df.loc[row, metrics[metric]])
    rv_df.loc[row, 'RV Score'] = mean(value_percentiles)
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,121.700,N/A,53.90,0.87,7.2800,0.7,7.2000,0.8,33.799598,0.88,13.074524,0.78,0.806
1,AAL,16.891,N/A,-1.30,0.08,-1.8500,0.04,0.4177,0.03,-6.154534,0.03,1.357425,0.07,0.05
2,AAP,169.900,N/A,24.05,0.47,2.9000,0.34,1.1400,0.18,11.298304,0.29,2.450171,0.17,0.29
3,AAPL,135.090,N/A,41.07,0.77,33.9200,0.98,8.0600,0.84,28.156315,0.83,20.291113,0.92,0.868
4,ABBV,109.510,N/A,22.87,0.44,12.4000,0.82,4.6000,0.63,15.968320,0.49,9.395932,0.62,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,CINF,85.323,N/A,18.00,0.31,1.3600,0.18,1.7700,0.25,7.778563,0.14,1.684011,0.1,0.196
96,CL,90.004,N/A,28.24,0.54,114.2400,0.99,4.5900,0.62,19.301071,0.61,7.936676,0.49,0.65
97,CLX,208.840,N/A,22.82,0.43,23.0000,0.95,3.7000,0.53,18.161695,0.56,8.015534,0.5,0.594
98,CMA,53.700,N/A,14.46,0.21,1.0016,0.1,2.3400,0.385,9.746571,0.22,2.713624,0.18,0.219


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [18]:
rv_df.sort_values('RV Score', ascending = True, inplace = True) #samller value, cheaper the stock
rv_df = rv_df[:50]
rv_df.reset_index(inplace=True, drop = True)
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,16.891,N/A,-1.30,0.08,-1.8500,0.04,0.4177,0.03,-6.154534,0.03,1.357425,0.07,0.05
1,AIG,39.940,N/A,-6.94,0.04,0.5334,0.06,0.7544,0.09,4.376271,0.06,0.703820,0.02,0.054
2,BA,236.530,N/A,-29.98,0.03,-11.2800,0.02,2.2000,0.33,-31.177332,0.01,-217.620205,0.01,0.08
3,ALL,110.940,N/A,7.50,0.12,1.3000,0.16,0.7401,0.08,2.422397,0.05,0.705849,0.03,0.088
4,C,60.630,N/A,12.12,0.16,0.7092,0.07,1.3300,0.2,6.666951,0.1,1.572077,0.09,0.124
5,AFL,46.241,N/A,11.57,0.15,1.1200,0.12,1.4600,0.21,6.157133,0.08,1.477695,0.08,0.128
6,APA,16.409,N/A,-0.76,0.09,-3.6900,0.03,1.1600,0.19,7.895398,0.15,2.787764,0.19,0.13
7,AIZ,135.370,N/A,20.20,0.37,1.3400,0.17,0.7814,0.1,1.884785,0.04,0.779356,0.04,0.144
8,BK,41.776,N/A,8.88,0.14,0.9138,0.09,1.9600,0.295,6.322025,0.09,1.957828,0.13,0.149
9,AIV,4.649,N/A,4.35,0.1,0.3540,0.05,0.8096,0.11,9.115649,0.19,5.535863,0.33,0.156


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [19]:
portfolio_input()

Enter the value of your portfolio:1000000


In [20]:
position_size = float(portfolio_size)/len(rv_df)
for i in range(0,len(rv_df)):
    rv_df.loc[i,'Number of Shares to Buy'] = math.floor(position_size/rv_df.loc[i, 'Price'])
    
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,16.891,1184,-1.30,0.08,-1.8500,0.04,0.4177,0.03,-6.154534,0.03,1.357425,0.07,0.05
1,AIG,39.940,500,-6.94,0.04,0.5334,0.06,0.7544,0.09,4.376271,0.06,0.703820,0.02,0.054
2,BA,236.530,84,-29.98,0.03,-11.2800,0.02,2.2000,0.33,-31.177332,0.01,-217.620205,0.01,0.08
3,ALL,110.940,180,7.50,0.12,1.3000,0.16,0.7401,0.08,2.422397,0.05,0.705849,0.03,0.088
4,C,60.630,329,12.12,0.16,0.7092,0.07,1.3300,0.2,6.666951,0.1,1.572077,0.09,0.124
5,AFL,46.241,432,11.57,0.15,1.1200,0.12,1.4600,0.21,6.157133,0.08,1.477695,0.08,0.128
6,APA,16.409,1218,-0.76,0.09,-3.6900,0.03,1.1600,0.19,7.895398,0.15,2.787764,0.19,0.13
7,AIZ,135.370,147,20.20,0.37,1.3400,0.17,0.7814,0.1,1.884785,0.04,0.779356,0.04,0.144
8,BK,41.776,478,8.88,0.14,0.9138,0.09,1.9600,0.295,6.322025,0.09,1.957828,0.13,0.149
9,AIV,4.649,4302,4.35,0.1,0.3540,0.05,0.8096,0.11,9.115649,0.19,5.535863,0.33,0.156


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [21]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine = 'xlsxwriter')
rv_df.to_excel(writer, 'value_strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [22]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    
    
    )

In [23]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['Price-to-Earnings Ratio', float_template],
    'E': ['PE Percentile', percent_template],
    'F': ['Price-to-Book Ratio', float_template],
    'G': ['PB Percentile', percent_template],
    'H': ['Price-to-Sales Ratio', float_template],
    'I': ['PS Percentile', percent_template],
    'J': ['EV/EBITDA', float_template],
    'K': ['EV/EBITDA Percentile', percent_template],
    'L': ['EV/GP', float_template],
    'M': ['EV/GP Percentile',percent_template],
    'N': ['RV Score',percent_template]
}

for column in column_formats.keys():
    writer.sheets['value_strategy'].set_column(f'{column}:{column}',18, column_formats[column][1])
    writer.sheets['value_strategy'].write(f'{column}1',column_formats[column][0],column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [24]:
writer.save()